In [1]:
from selenium import webdriver
from PIL import Image
import os
from os import path
import time
import random
import pandas as pd
import requests
import json
import numpy as np

In [1]:
import code.visualizations as viz
import code.data_preparation as dprep

In [2]:
api_keys = dprep.get_api_keys()
omdb_key = api_keys[0]
imdb_key = api_keys[1]
tmdb_key = api_keys[2]


## Scraping functions

In [4]:
# scrape financial details from the numbers website, 
# make numbers_df, clean the data and filter for only last 15 years

financials_list = dprep.the_numbers_scraping()

In [5]:
# make numbers_df, filter out corrupt/anomolous rows by rejecting movies
# with no worldwide takings and movies with budget <$1m 
# only include movies with presence in the US

numbers_df = dprep.construct_prepare_numbers_df(financials_list)

In [6]:
# make financials_df, copy of number_df with cleaned data, filtered for only
# last 15 years and empty column added to hold imdb_code

financials_15_df = dprep.make_fin_15(numbers_df)


In [7]:
# use omdb api to get attributes for movies in numbers_df
# create a new df that is a copy of numbers_df but with the imdb_code, which 
# will be used as the primary key, filled in. Do this so we have a separate df
# after each time-consuming scrape/api session, so it easier to isolate different
# blocs of code to work with
# the function call also creates a global dictionary of the omdb attributes that
# will be used to create the attributes database

(omdb_attrs, financials_15_df) = dprep.make_omdb_dict(financials_15_df, omdb_key)

C:\Users\james\Documents\flatiron_data_science\phase1\Project_fin\dsc-phase-1-project\code\data_preparation.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financials_15_df['imdb_code'][financials_15_df.title == movie] = imdb_id


In [8]:
# manual imdb code insertion for problematic titles
financials_15_df = dprep.update_make_omdb_dict(financials_15_df, omdb_key)

In [9]:
# makes a dataframe of movie attributes using omdb api, imdb key is primary key
attributes_df = dprep.make_attributes_df()

## Save the dataframes

In [10]:
dprep.save_files(financials_list, numbers_df, financials_15_df, omdb_attrs, attributes_df)

**DATAFRAMES** <br><br>
**numbers_df** - all financial data from The Numbers website<br>
**financials_15_df** - all financial data for numbers movies in last 15 years <br>
**financials_hits_df** - same data but filtered out loss making movies, those only succesful outside english speaking world and those with no documented figures <br>
**financials_flops_df** - same but filtered out profit making movies <br>
<br>

In [ ]:
manual_list = ['Star Wars Ep. VII: The Force Awakens', 
'Star Wars Ep. VIII: The Last Jedi', 
'Prince of Persia: Sands of Time', 
'Fast and Furious 6', 
'The Chronicles of Narnia: The Voyage of the Daw…', 
'Harry Potter and the Deathly Hallows: Part II', 
'Harry Potter and the Deathly Hallows: Part I', 
'Fantastic Four: Rise of the Silver Surfer', 
'The Hangover 3', 
'Ford v. Ferrari', 
'Dr. Seuss’ The Grinch', 
'Mamma Mia: Here We Go Again!', 
'The Angry Birds Movie', 
'Wall Street 2: Money Never Sleeps',
'Dr. Seuss’ The Lorax', 
'All Eyez on Me', 
'John Wick: Chapter Two', 
'Underworld 3: Rise of the Lycans', 
'Halloween 2',        
'Precious (Based on the Novel Push by Sapphire)']

manual_imdb = ['tt2488496', 'tt2527336', 'tt0473075', 'tt1905041', 'tt0980970', 'tt1201607', 'tt0926084', 'tt0486576', 'tt1951261', 'tt1950186', 'tt2709692', 'tt6911608', 'tt1985949', 'tt1027718', 'tt1482459', 'tt1666185', 'tt4425200', 'tt0834001', 'tt1311067', 'tt0929632']
